<a href="https://colab.research.google.com/github/Sarah-0405/Bachelorarbeit_/blob/main/CORINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap
import folium
import geopandas as gpd
from google.colab import files
import zipfile
import os


In [ ]:
ee.Authenticate()
ee.Initialize(project="ee-sarahheinz0405")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
clc = ee.Image("COPERNICUS/CORINE/V20/100m/2018")


In [ ]:
print(clc.getInfo())

{'type': 'Image', 'bands': [{'id': 'landcover', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [65000, 46000], 'crs': 'EPSG:3035', 'crs_transform': [100, 0, 900000, 0, -100, 5500000]}], 'version': 1578598451411424, 'id': 'COPERNICUS/CORINE/V20/100m/2018', 'properties': {'system:time_start': 1483228800000, 'landcover_class_names': ['Artificial surfaces; urban fabric; continuous urban fabric', 'Artificial surfaces; urban fabric; discontinuous urban fabric', 'Artificial surfaces; industrial, commercial, and transport units; industrial or commercial units', 'Artificial surfaces; industrial, commercial, and transport units; road and rail networks and associated land', 'Artificial surfaces; industrial, commercial, and transport units; port areas', 'Artificial surfaces; industrial, commercial, and transport units; airports', 'Artificial surfaces; mine, dump, and construction sites; mineral extraction sites', 'Artificial surfaces; mine, dump,

In [ ]:
# Alle Landnutzungsklassen abrufen
klassen = clc.get('landcover_class_names').getInfo() # Use .getInfo() to retrieve the data

print(klassen)

['Artificial surfaces; urban fabric; continuous urban fabric', 'Artificial surfaces; urban fabric; discontinuous urban fabric', 'Artificial surfaces; industrial, commercial, and transport units; industrial or commercial units', 'Artificial surfaces; industrial, commercial, and transport units; road and rail networks and associated land', 'Artificial surfaces; industrial, commercial, and transport units; port areas', 'Artificial surfaces; industrial, commercial, and transport units; airports', 'Artificial surfaces; mine, dump, and construction sites; mineral extraction sites', 'Artificial surfaces; mine, dump, and construction sites; dump sites', 'Artificial surfaces; mine, dump, and construction sites; construction sites', 'Artificial surfaces; artificial, non-agricultural vegetated areas; green urban areas', 'Artificial surfaces; artificial, non-agricultural vegetated areas; sport and leisure facilities', 'Agricultural areas; arable land; non-irrigated arable land', 'Agricultural area

In [ ]:
# 4. Definition der CORINE-Gruppierung und Farbpalette
gruppen = {
    'Urbane Strukturen': [111, 112],
    'Industriegebiete': [121, 122, 123, 124],
    'Verkehrslinien': [131, 132, 133],
    'urbane Grünflaechen': [141, 142],
    'Waelder': [311, 312, 313, 321, 322, 323, 324],
    'Gewaesser': [511, 512, 521, 522, 523],
    'Landwirtschaft': [211, 242, 243, 231, 221, 222, 223, 241, 212, 213, 244]
}

farben = {
    'Urbane Strukturen': '#F5F5DC',  # beige
    'Industriegebiete': '#696969', # dunkelgrau
    'Verkehrslinien': '#D3D3D3', #dunkelgrau
    'urbane Grünflaechen': '#90EE90',  #hellgrün
    'Waelder': '#006400',  # dunklegrün
    'Gewaesser': '#0000FF',  #blau
    'Landwirtschaft': '#FFA07A' #lachs
}

# Erstelle Listen für Remapping: Für jeden CORINE-Code wird ein Gruppenindex vergeben.
# Wir ordnen den Gruppen fortlaufend die IDs 1 bis 7 zu.
group_order = ['Urbane Strukturen', 'Industriegebiete', 'Verkehrslinien',
               'urbane Grünflaechen', 'Waelder', 'Gewaesser', 'Landwirtschaft']

old_values = []
new_values = []
for i, group in enumerate(group_order, start=1):
    codes = gruppen[group]
    for code in codes:
        old_values.append(code)
        new_values.append(i)

# Palette in derselben Reihenfolge wie group_order:
palette = [farben[group] for group in group_order]

# 5. Funktion: Shapefile (mittels GeoPandas) in ein EE-FeatureCollection umwandeln
def shapefile_to_ee_fc(shp_path):
    # Lese das Shapefile ein und reprojiziere nach EPSG:4326 (WGS84)
    gdf = gpd.read_file(shp_path)
    gdf = gdf.to_crs(epsg=4326)
    features = []
    for _, row in gdf.iterrows():
        geom = ee.Geometry(row['geometry'].__geo_interface__)
        # Optional: Füge einen Eigenschaftswert (z.B. "name") hinzu, falls vorhanden
        feature = ee.Feature(geom)
        features.append(feature)
    return ee.FeatureCollection(features)

# 6. Lade die Shapefiles für Nördlingen, Kempten und Ingolstadt aus Google Drive
noerdlingen_shapefile = '/content/drive/MyDrive/Daten BA /Shapefile_Zensus2022/EPSG_25832/Nördlingen_Gerenze_shp.shp'
kempten_shapefile   = '/content/drive/MyDrive/Daten BA /Shapefile_Zensus2022/EPSG_25832/Kempten (Allgäu) .shp'
ingolstadt_shapefile = '/content/drive/MyDrive/Daten BA /Shapefile_Zensus2022/EPSG_25832/Ingolstadt .shp'

noerdlingen_fc = shapefile_to_ee_fc(noerdlingen_shapefile)
kempten_fc     = shapefile_to_ee_fc(kempten_shapefile)
ingolstadt_fc  = shapefile_to_ee_fc(ingolstadt_shapefile)

# Optional: Alle Regionen in einer einzigen FeatureCollection zusammenführen
region_fc = noerdlingen_fc.merge(kempten_fc).merge(ingolstadt_fc)

# 7. CORINE Land Cover-Daten laden und gruppieren
clc = ee.Image("COPERNICUS/CORINE/V20/100m/2018")
# Die CORINE-Bandbezeichnung ist "landcover". Mithilfe von remap ordnen wir den Pixeln die Gruppen-IDs zu.
clc_grouped = clc.select('landcover').remap(old_values, new_values, 0)  # 0 als Default-Wert

# 8. Visualisierungseinstellungen für die gruppierten Daten
group_vis = {
    'min': 1,
    'max': 7,
    'palette': palette
}

# 9. Erstelle eine interaktive Karte und visualisiere:
Map = geemap.Map(center=[48.7, 10.5], zoom=8)
# Zeige das gruppierte CORINE-Bild, auf die kombinierte Region (Städtegrenzen) zugeschnitten
Map.addLayer(clc_grouped.clip(region_fc), group_vis, "CLC Gruppen")
# Optional: Zeige die Umrisse der Shapefiles (Städtegrenzen) in schwarz
Map.addLayer(region_fc.style(**{'color': 'black', 'fillColor': '00000000', 'width': 0.5}), {}, "Städtegrenzen")
Map.add_layer_control()  # Layer-Steuerelement hinzufügen


# Legende hinzufügen
legend_dict = {group: farben[group] for group in gruppen}  # Create legend_dict with group names and colors

Map.add_legend(legend_dict=legend_dict, title="Landnutzungsklassen", position='topleft')  # Add legend with group names
Map


Map(center=[48.7, 10.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [ ]:
# Definiere die Region, für die du die Daten extrahieren möchtest (z.B. region_fc)
region = region_fc

# Extrahiere die Landnutzungsdaten für die Region aus dem gruppierten CORINE-Bild
landnutzung = clc_grouped.sample(region=region, scale=100, geometries=True) # scale=100 entspricht der Auflösung des CORINE-Bildes

In [ ]:
noerdlingen_path = '/content/drive/MyDrive/Daten BA /erstellte Dateien mit colab/LST_Noerdlingen_summer2022_5%_with_coords_reexported.csv'
kempten_path = '/content/drive/MyDrive/Daten BA /erstellte Dateien mit colab/LST_Kempten_summer2022_5%_with_coords.csv'
ingolstadt_path = '/content/drive/MyDrive/Daten BA /erstellte Dateien mit colab/LST_Ingolstadt_summer2022_5%_with_coords.csv'

In [ ]:
# csv-Dateien mit LST Werten in ee.Images umwandeln
import rasterio
import pandas as pd

# Function to convert CSV to GeoTIFF and upload to Earth Engine
def csv_to_ee_image(csv_path, band_name='LST_Celsius'):
    """Converts a CSV file with coordinates and LST values to a GeoTIFF and uploads it to Earth Engine.

    Args:
        csv_path (str): Path to the CSV file.
        band_name (str, optional): Name of the band in the GeoTIFF. Defaults to 'LST'.

    Returns:
        ee.Image: Earth Engine Image object representing the GeoTIFF.
    """
    # 1. Load the CSV data using pandas
    df = pd.read_csv(csv_path)

    # 2. Create a GeoDataFrame from the pandas DataFrame
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs='EPSG:4326')  # Assuming 'longitude' and 'latitude' columns in your CSV

    # 3. Define the output GeoTIFF file path
    geotiff_path = csv_path.replace('.csv', '.tif')

    # 4. Create a new GeoTIFF using rasterio
    with rasterio.open(
        geotiff_path,
        'w',
        driver='GTiff',
        height=gdf.shape[0],
        width=1,  # Single-band image
        count=1,
        dtype=rasterio.float32,
        crs='EPSG:4326',
        transform=rasterio.transform.from_bounds(gdf.total_bounds[0], gdf.total_bounds[1], gdf.total_bounds[2], gdf.total_bounds[3], 1, 1) # Update bounds and resolution as needed
    ) as dst:
        dst.write(gdf[band_name].values.reshape(1, -1), 1)  # Write LST values to the GeoTIFF


    return ee.Image(geotiff_path)

# Call the function to convert and load your CSV files:
noerdlingen_image = csv_to_ee_image(noerdlingen_path)
kempten_image = csv_to_ee_image(kempten_path)
ingolstadt_image = csv_to_ee_image(ingolstadt_path)


# LST-csv Dateien mit Landnutzungsdaten kombinieren

In [ ]:
# Function to convert CSV to GeoDataFrame
def csv_to_gdf(csv_path):
    """Converts a CSV file with coordinates and LST values to a GeoDataFrame.

    Args:
        csv_path (str): Path to the CSV file.

    Returns:
        gpd.GeoDataFrame: GeoDataFrame object representing the CSV data.
    """
    df = pd.read_csv(csv_path)
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs='EPSG:4326')
    return gdf

# Load the CSV files as GeoDataFrames
noerdlingen_gdf = csv_to_gdf(noerdlingen_path)
kempten_gdf = csv_to_gdf(kempten_path)
ingolstadt_gdf = csv_to_gdf(ingolstadt_path)


# Lade die LST-Daten und Städtegrenzen für jede Stadt einzeln
cities = {
       'noerdlingen': {
           'lst_gdf': csv_to_gdf(noerdlingen_path),
           'region_fc': noerdlingen_fc
       },
       'kempten': {
           'lst_gdf': csv_to_gdf(kempten_path),
           'region_fc': kempten_fc
       },
       'ingolstadt': {
           'lst_gdf': csv_to_gdf(ingolstadt_path),
           'region_fc': ingolstadt_fc
       }
   }

# Berechnungen für jede Stadt durchführen
for city_name, city_data in cities.items():
       print(f"Verarbeitung von {city_name}...")

       # LST-Daten in ee.FeatureCollection umwandeln
       lst_fc = geemap.geopandas_to_ee(city_data['lst_gdf'])

       # Landnutzungsdaten für die Stadt extrahieren
       landnutzung_daten = clc_grouped.reduceRegions(
           collection=lst_fc,
           reducer=ee.Reducer.first(),
           scale=100,
           tileScale=16
       )


# Die FeatureCollection in kleinere Batches aufteilen
batch_size = 1000  # Anzahl der Features pro Batch, anpassen bei Bedarf
num_batches = landnutzung_daten.size().getInfo() // batch_size + 1

for i in range(num_batches):
        # Den aktuellen Batch extrahieren
        batch = landnutzung_daten.toList(batch_size, i * batch_size)
        batch_fc = ee.FeatureCollection(batch)

        # Den Batch als GeoDataFrame exportieren
        landnutzung_gdf = geemap.ee_export_vector(batch_fc, filename=f"landnutzung_daten_{city_name}_batch_{i}.shp")

        # Spatial Join mit den LST-Daten durchführen
        combined_gdf = gpd.sjoin(city_data['lst_gdf'], landnutzung_gdf, how='inner', op='intersects')

        # Ergebnis als CSV exportieren
        combined_gdf.to_csv(f"combined_lst_landcover_{city_name}_batch_{i}.csv", index=False)

print("Fertig!")

Verarbeitung von noerdlingen...
Verarbeitung von kempten...
Verarbeitung von ingolstadt...


EEException: Request payload size exceeds the limit: 10485760 bytes.

In [ ]:
# CRS überprüfen
print(noerdlingen_image.projection().getInfo())
print(clc_grouped.projection().getInfo())  # CORINE-Daten

# Auflösung überprüfen
print(noerdlingen_image.projection().nominalScale().getInfo())
print(clc_grouped.projection().nominalScale().getInfo())  # CORINE-Daten

# Wiederhole die Überprüfung für kempten_image und ingolstadt_image

In [ ]:
# Optional
# Erstelle eine Liste der LST-Images
lst_images = [noerdlingen_image, kempten_image, ingolstadt_image]

# Erstelle ein Mosaik aus den LST-Images
lst_mosaic = ee.ImageCollection(lst_images).mosaic()

In [ ]:
# Definiere die Region, für die du die Daten extrahieren möchtest (z.B. region_fc)
region = region_fc

# Extrahiere die LST-Werte für die Region aus dem LST-Mosaik
lst_samples = lst_mosaic.sampleRegions(
    collection=region,
    properties=['LST_Celsius'],  # Die Eigenschaft, die du extrahieren möchtest - hier der Bandname aus deinem GeoTIFF
    scale=100,  # Die Auflösung, in der du die Daten extrahieren möchtest
    geometries=True  # Gibt die Geometrien der Pixel zurück
)

In [ ]:
landnutzung_samples = clc_grouped.sampleRegions(
    collection=region,
    properties=['landcover'],
    scale=100,
    geometries=True
)

In [ ]:
import geopandas as gpd

# Export the FeatureCollections to GeoDataFrames
lst_gdf = geemap.ee_export_vector(lst_samples, filename="lst_samples.shp")  # Export lst_samples
landnutzung_gdf = geemap.ee_export_vector(landnutzung_samples, filename="landnutzung_samples.shp")  # Export landnutzung_samples

# Perform a spatial join to merge the properties
combined_gdf = gpd.sjoin(lst_gdf, landnutzung_gdf, how='inner', op='intersects')

# Convert the combined GeoDataFrame back to an ee.FeatureCollection
combined_fc = geemap.geopandas_to_ee(combined_gdf)

# Export the combined FeatureCollection to a CSV file
df = geemap.ee_export_vector(combined_fc, filename="combined_data.csv")